In [ ]:
# Demonstracao

In [1]:
import json
import string
import pandas as pd
import warnings

%matplotlib inline
warnings.filterwarnings("ignore")

In [31]:
#Parametros Gerais
df_parametros_geral = pd.read_excel('online/parametros_Gerais.xlsx', sep=",")
df_parametros_geral.head(10)

,Empresa,ValorMaximo
0,PORTO,25
1,AZUL,35


In [138]:
#Parametros - Classe / Empresa
df_parametros = pd.read_excel('online/parametros_classe.xlsx', sep=",")
df_parametros.head(10)

,Empresa,classeLocalizacao,ValorMaximo
0,PORTO,1,10
1,AZUL,1,15
2,PORTO,11,20
3,AZUL,11,20


In [5]:
#Load dos modelos - Churn e Sinistro
from sklearn.externals import joblib

modelo_churn = joblib.load('modelos/gbm_modeloChurn.joblib')
modelo_sinistro = joblib.load('modelos/gbm_modeloSinistro.joblib')

In [7]:
modelo_sinistro

LGBMClassifier(boosting_type='gbdt', class_weight=None,
        colsample_bytree=0.55607546409401, learning_rate=0.04,
        max_depth=-1, min_child_samples=103, min_child_weight=10.0,
        min_split_gain=0.0, n_estimators=1500, n_jobs=-1, num_leaves=100,
        objective=None, random_state=None, reg_alpha=2, reg_lambda=1,
        silent=True, subsample=0.8939112927620336,
        subsample_for_bin=200000, subsample_freq=1)

In [57]:
# LOAD Arquivos de Predição
df_full = pd.read_excel('online/simulador.xlsx')
df_churn = df_full[["cliente","empresa","classeLocalizacao","anosFidelidadeCliente","codigoTipoVeiculo","codigoSucursal","premioLiquidoPagoApolice","valorFranquia","valorPremioFinal","valorDiferencaPremioAnual","valorImportanciaSeguradaCasco","origemProposta","codigoClasseBonus","valorPremioPagoAtual"]]
df_sinistro = df_full[["cliente","empresa","classeLocalizacao","IDHM_R","anosFidelidadeCliente","codigoClasseBonus","codigoClasseLocalizacao","codigoFamiliaVeiculo","codigoMarcaVeiculo","codigoSucursal","codigoTipoVeiculo","diaSemana","diferencaPremioAntNovo","premioLiquidoPagoApolice","valorDiferencaPremioAnual","valorFranquia","valorImportanciaSeguradaCasco","valorPremioFinal","valorPremioPagoAtual"]]

df_sinistro.head(10)


,cliente,empresa,classeLocalizacao,IDHM_R,anosFidelidadeCliente,codigoClasseBonus,codigoClasseLocalizacao,codigoFamiliaVeiculo,codigoMarcaVeiculo,codigoSucursal,codigoTipoVeiculo,diaSemana,diferencaPremioAntNovo,premioLiquidoPagoApolice,valorDiferencaPremioAnual,valorFranquia,valorImportanciaSeguradaCasco,valorPremioFinal,valorPremioPagoAtual
0,Cliente 1,PORTO,1,0.729,11,10,11,35,1,44,64,3,-346.355,2440.96,553.0900,2523,31802.00,2498.785,2152.43
1,Cliente 2,PORTO,10,0.729,11,10,10,25,59,36,20,3,824.380,1765.76,1562.4100,3202,25085.00,1328.050,2152.43
2,Cliente 3,PORTO,10,0.756,11,10,10,25,59,36,20,2,824.380,1765.76,1562.4100,3202,25085.00,1328.050,2152.43
3,Cliente 4,PORTO,11,0.789,12,10,11,3,3,36,136,3,367.730,2072.34,-245769.0000,753,53697.00,1764.150,2131.88
4,Cliente 5,PORTO,13,0.756,5,2,13,2,130,68,1,3,385.870,1449.71,1167.6300,2874,31974.00,1109.240,1495.11
5,Cliente 6,PORTO,26,0.729,3,1,26,18,11,18,7,4,83.130,1370.37,1245.0900,2235,13728.00,1245.090,1328.22
6,Cliente 7,PORTO,11,0.782,2,8,11,35,1,40,86,4,1626.770,1676.89,-101.0400,1820,81562.95,-33.830,1592.94
7,Cliente 8,PORTO,38,0.756,2,2,38,30,64,6,121,4,-21.740,3104.99,3223.4400,4572,63871.00,2868.860,2847.12
8,Cliente 9,AZUL,1,0.720,2,4,1,35,3,15,147,2,2739.720,3341.97,5734.1351,1921,51881.00,404.610,3144.33
9,Cliente 10,AZUL,11,0.756,4,5,11,3,1,60,59,5,1131.550,2454.26,2030.7400,3092,30363.00,1827.670,2959.22


In [63]:
#Fazendo as previsões
array_churn = df_churn.values
X = array_churn[:,3:14]

array_sinistro = df_sinistro.values
X_sinistro = array_sinistro[:,3:19]


In [64]:
#Normalizacao e Padronizacao dos dados
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_sinistro = scaler.fit_transform(X_sinistro)


In [65]:
X

array([[ 1.13294134, -0.04050697,  0.44288753,  0.46144575, -0.0986607 ,
         1.28791166,  0.31532006, -0.45642975,  1.        ,  1.05555556,
        -0.07274471],
       [ 1.13294134, -0.88922433,  0.00546775, -0.58881119,  0.59626271,
        -0.12824169,  0.32890217, -0.79322794, -1.        ,  1.05555556,
        -0.07274471],
       [ 1.13294134, -0.88922433,  0.00546775, -0.58881119,  0.59626271,
        -0.12824169,  0.32890217, -0.79322794, -1.        ,  1.05555556,
        -0.07274471],
       [ 1.37399268,  1.34830327,  0.00546775, -0.11193356, -1.91016944,
         0.3992769 , -2.99936218,  0.64141103, -1.        ,  1.05555556,
        -0.10736419],
       [-0.31336675, -1.25571592,  1.75514687, -1.08041916,  0.26057069,
        -0.3929203 ,  0.32358974, -0.44780546, -1.        , -1.16666667,
        -1.18009642],
       [-0.79546945, -1.13998174, -0.97872676, -1.20383057, -0.39341467,
        -0.2285924 ,  0.32463209, -1.36268113, -1.        , -1.44444444,
        -1.461

In [67]:
Y_churn_prob = modelo_churn.predict_proba(X, num_iteration=modelo_churn.best_iteration_)
Y_sinistro_prob = modelo_sinistro.predict_proba(X_sinistro, num_iteration=modelo_sinistro.best_iteration_)
Y_churn = modelo_churn.predict(X, num_iteration=modelo_churn.best_iteration_)
Y_sinistro = modelo_sinistro.predict(X_sinistro, num_iteration=modelo_sinistro.best_iteration_)



In [84]:
df_resultado = df_full[["cliente","empresa","classeLocalizacao"]]
df_resultado['Vai_Renovar'] = Y_churn
df_resultado['Ter_Sinistro'] = Y_sinistro
df_resultado['Prob_Nao_Renovacao'] = (Y_churn_prob[:,0] * 100)
df_resultado['Prob_Ter_Sinistro'] = (Y_sinistro_prob[:,1] * 100)

df_resultado['Vai_Renovar'] = ['Sim' if x == 1.0 else  'Nao' for x in df_resultado['Vai_Renovar'] ]
df_resultado['Ter_Sinistro'] = ['Sim' if x == 1.0 else  'Nao' for x in df_resultado['Ter_Sinistro'] ]


In [175]:
# 100 - Prob_Ter_Sinistro (Percentual de Desconto)
# 35 * (Percentual de Desconto / 100)
# Se abaixo de 10% => Desconto Maximo

# Se Renova Nao e Sinistro Nao - Desconto Maior
# Se Renova Nao e Sinistro Sim - Desconto Menor
# Se Renova Sim e Sinistro Nao - Desconto 0%
# Se Renova Sim e Sinistro Sim - Aumento o preço

desconto_max_Porto = df_parametros_geral.iloc[0,1].astype(float)
desconto_max_Azul = df_parametros_geral.iloc[1,1].astype(float)

lista_classes_Porto = df_parametros[df_parametros.Empresa == 'PORTO'].iloc[:,1:3].values
lista_classes_Azul = df_parametros[df_parametros.Empresa == 'AZUL'].iloc[:,1:3].values

def calcularDesconto(empresa, classe, vaiRenovar, terSinistro, probNaoRenovacao, probTerSinistro): 
    
        desconto_maximo = desconto_max_Porto if (empresa == 'PORTO') else desconto_max_Azul
        if (vaiRenovar == 'Nao'):
            return (desconto_maximo * ((100 - probTerSinistro) / 100) )
        else: 
            return 0.0
            

def calcularAumento(empresa, classe, vaiRenovar, terSinistro, probNaoRenovacao, probTerSinistro): 
        
        aumento_maximo = desconto_max_Porto if (empresa == 'PORTO') else desconto_max_Azul
        if (vaiRenovar == 'Sim'):
            if (terSinistro == 'Sim'): 
                return (aumento_maximo * ( (probTerSinistro - 50.0) / 100 ) )
            else: 
                return 0.0
        else: 
            return 0.0


In [176]:
df_resultado['desconto'] = [ calcularDesconto(e,c,x,y,z,m) for e,c,x,y,z,m in df_resultado.iloc[:,1:7].values ]
df_resultado['aumento'] = [ calcularAumento(e,c,x,y,z,m) for e,c,x,y,z,m in df_resultado.iloc[:,1:7].values ]


In [177]:
df_resultado.head(10)

,cliente,empresa,classeLocalizacao,Vai_Renovar,Ter_Sinistro,Prob_Nao_Renovacao,Prob_Ter_Sinistro,descontox,aumentox,desconto,aumento
0,Cliente 1,PORTO,1,Nao,Sim,85.346130,57.349584,5.0,Sim,10.662604,0.000000
1,Cliente 2,PORTO,10,Nao,Nao,90.500334,13.141508,10.0,Nao,21.714623,0.000000
2,Cliente 3,PORTO,10,Nao,Nao,90.500334,20.148807,10.0,Nao,19.962798,0.000000
3,Cliente 4,PORTO,11,Sim,Sim,6.628218,76.769598,0.0,Sim,0.000000,6.692400
4,Cliente 5,PORTO,13,Nao,Nao,55.376378,14.635433,10.0,Nao,21.341142,0.000000
5,Cliente 6,PORTO,26,Nao,Nao,55.758482,5.169866,10.0,Nao,23.707533,0.000000
6,Cliente 7,PORTO,11,Sim,Sim,30.844425,67.451250,0.0,Sim,0.000000,4.362812
7,Cliente 8,PORTO,38,Sim,Nao,25.884706,44.304292,0.0,Nao,0.000000,0.000000
8,Cliente 9,AZUL,1,Sim,Sim,31.112904,78.867718,0.0,Sim,0.000000,10.103701
9,Cliente 10,AZUL,11,Nao,Sim,96.437241,72.860965,5.0,Sim,9.498662,0.000000


In [91]:
df_resultado.to_excel('resultado/resultado.xlsx', header=True, index=False, engine='openpyxl')

In [ ]:
# 100 - Prob_Ter_Sinistro (Percentual de Desconto)
# 35 * (Percentual de Desconto / 100)
# Se abaixo de 10% => Desconto Maximo

# Se Renova Nao e Sinistro Nao - Desconto Maior
# Se Renova Nao e Sinistro Sim - Desconto Menor
# Se Renova Sim e Sinistro Nao - Desconto 0%
# Se Renova Sim e Sinistro Sim - Aumento o preço

